In [ ]:
from azure.search.documents.indexes import SearchIndexClient
from azure.core.credentials import AzureKeyCredential
import os


azure_search_key = os.getenv('AZURE_SEARCH_KEY')
azure_search_endpoint = os.getenv('AZURE_SEARCH_ENDPOINT')

credential = AzureKeyCredential(azure_search_key)
index_client = SearchIndexClient(endpoint=azure_search_endpoint, credential=credential)

TypeError: key must be a string.

In [ ]:
from azure.search.documents import SearchClient
from azure.search.documents.indexes.models import (
    ComplexField,
    SimpleField,
    SearchFieldDataType,
    SearchableField,
    SearchIndex
)

fields = [
    SimpleField(name="DocId", type=SearchFieldDataType.String, key=True),
    SearchableField(name="Exam", type=SearchFieldDataType.String),
    SearchableField(name="Module", type=SearchFieldDataType.String),
    SearchableField(name="Topic", type=SearchFieldDataType.String),
    SearchableField(name="content", type=SearchFieldDataType.String),
    #images
    #urls
    #
    # vector_embeddigs = 


]

index = SearchIndex(name="exam-simulator-idx", fields=fields)
result = index_client.create_or_update_index(index)
print(result.name)

exam-simulator-idx


In [6]:
documents = [
    {
        "DocId":"1",
        "Exam":"AI-102 AI Engineer",
        "Module": "Develop generative AI apps in Azure",
        "Topic": "Plan and prepare to develop AI solutions on Azure",
        "content": """Introduction
Completed
100 XP
1 minute
The growth in the use of artificial intelligence (AI) in general, and generative AI in particular means that developers are increasingly required to create comprehensive AI solutions. These solutions need to combine machine learning models, AI services, prompt engineering solutions, and custom code.

Microsoft Azure provides multiple services that you can use to create AI solutions. However, before embarking on an AI application development project, it's useful to consider the available options for services, tools, and frameworks as well as some principles and practices that can help you succeed.

This module explores some of the key considerations for planning an AI development project, and introduces Azure AI Foundry; a comprehensive platform for AI development on Microsoft Azure."""
    }
]

search_index_client = SearchClient(endpoint=azure_search_endpoint, index_name="exam-simulator-idx", credential=credential)

result = search_index_client.upload_documents(documents=documents)

In [14]:
results = search_index_client.search(
    query_type="simple",
    search_text="*",
    select = 'Module,Topic,content',
    include_total_count=True
)

for k in results:
    print(k['Topic'])
    print(k['Module'])
    print(k['@search.score'])
    print(k['content'])

Plan and prepare to develop AI solutions on Azure
Develop generative AI apps in Azure
1.0
Introduction
Completed
100 XP
1 minute
The growth in the use of artificial intelligence (AI) in general, and generative AI in particular means that developers are increasingly required to create comprehensive AI solutions. These solutions need to combine machine learning models, AI services, prompt engineering solutions, and custom code.

Microsoft Azure provides multiple services that you can use to create AI solutions. However, before embarking on an AI application development project, it's useful to consider the available options for services, tools, and frameworks as well as some principles and practices that can help you succeed.

This module explores some of the key considerations for planning an AI development project, and introduces Azure AI Foundry; a comprehensive platform for AI development on Microsoft Azure.
